Here's production-level code using LangChain + Pydantic to build a smart tool that can run on AWS Lambda (or any cloud function), with structured validation, tool agent, and OpenAI integration.

✅ Use Case: Get Weather + Currency Conversion (2 Tools)

In [ ]:
!pip install langchain openai pydantic

In [ ]:
import os
from typing import Optional
from pydantic import BaseModel, Field
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.llms import OpenAI
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# LLM Initialization
llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0)


In [ ]:
# Tool 1: Weather (Mocked for demo)
class WeatherInput(BaseModel):
    location: str = Field(..., description="City name")

def get_weather(location: str) -> str:
    return f"Current weather in {location} is 30°C and sunny."


In [ ]:
weather_tool = Tool.from_function(
    func=get_weather,
    name="get_weather",
    description="Get current weather by location",
    args_schema=WeatherInput
)

In [ ]:
# Tool 2: Currency Converter
class CurrencyInput(BaseModel):
    amount: float = Field(..., description="Amount to convert")
    from_currency: str = Field(..., description="Source currency code (e.g., USD)")
    to_currency: str = Field(..., description="Target currency code (e.g., INR)")

def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    rate = 83.0  # Mock conversion rate for demo
    converted = amount * rate
    return f"{amount} {from_currency} = {converted:.2f} {to_currency}"


In [ ]:
currency_tool = Tool.from_function(
    func=convert_currency,
    name="convert_currency",
    description="Convert currency between two types",
    args_schema=CurrencyInput
)


In [ ]:
# Initialize Agent
tools = [weather_tool, currency_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [ ]:
# Handler Function (for AWS Lambda or Cloud Function)
def lambda_handler(event: dict, context: Optional[dict] = None) -> dict:
    query = event.get("query") or "What's the weather in Bangalore and convert 100 USD to INR?"
    result = agent.run(query)
    return {"response": result}